# Retrieval-Augmented Generation (RAG)

A type of AI language model architecture that combines the strengths of traditional transformer-based language models with the strengths of retrieval-based approaches.

## 1st step: **Indexing**
1. Load: First we need to load our data. This is done with DocumentLoaders.
2. Split: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won’t fit in a model’s finite context window.
3. Store: We need somewhere to store and index our splits, so that they can later be searched over. This is often done using a VectorStore and Embeddings model.

![Rag Architecture](../Resources/RAG_image1.png)

## 2nd Step: **Query**

![Rag Query](../Resources/RAG_image2.png)

## Traditional Generation vs RAG

**Traditional Generation**

* Generates text entirely from scratch
* Uses learnable parameters and mathematical operations
* Can be effective for generating coherent and fluent text, but may not always produce highly relevant or accurate results

**Retrieval-Augmented Generation (RAG)**

* Augments traditional generation with a retrieval step
* Retrieves relevant pieces of text from a large database
* Combines retrieved text with generation to inform the text generation process

## Benefits of RAG

* Benefits from the coherence and fluency of traditional generation-based models
* Leverages the accuracy and relevance of retrieval-based approaches
* Generates more accurate and informative text by incorporating external knowledge

## Applications of RAG

* Text summarization
* Question answering
* Conversation generation
* Chatbots
* Content creation
* Language translation

In [1]:

from typing import Any
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores.faiss import FAISS

retriever: Any = None

def create_in_memory_retriever(chunks:list):
    embed_function = OpenAIEmbeddings(model="text-embedding-ada-002")
    # Create a FAISS vector store from the document chunks
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embed_function
    )
    if retriever is not None:
        print("A retriever already exists")
        return retriever
    global retriever
    retriever = vectorstore.as_retriever()

def rag(question:str, retriever: Any):
    local_llm = ChatOpenAI(name="gpt-3.5-turbo")
    prompt_template="""Answer the question based only on the following context:
    ==============================
    {context}
    ==============================
    Question: {question}"""
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = (
        # You need a RunnablePassThrough if you are going to pass the parameter to the chain
        {
            "context": lambda x: retriever.invoke(x["question"]),
            "question": RunnablePassthrough(),
            "result": (
                lambda x: print(f"Question: {x['question']}\nContext: {x['context']}"),
                StrOutputParser(),
                local_llm,
                prompt
            )
        }
    )
    result = chain.invoke({"question": question})
    print(f"Question: {question}\nContext: {result['context']}")
    return result["result"]

ImportError: module ''langchain_core._api'.'deprecation'' not found

In [ ]:
from langchain_core.documents import Document
text = """
The astronauts on the International Space Station conducted a spacewalk to repair a malfunctioning solar panel. 
The aroma of freshly baked croissants wafted through the charming French bakery. 
The new policy aimed to reduce carbon emissions by 50% within the next decade. 
The ancient Egyptian pharaohs were known for their elaborate headdresses and ornate jewelry."""

# Manual Splitting
chunks = []
chunk_size = 35 # Characters
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]
print(documents)

In [ ]:
rag("What the astrounauts did ?")